# 07_데이터 정제 및 준비



## 7.1 누락된 데이터 처리하기



In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
string_data[0] = None

In [6]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

> NA 처리 메서드
> - dorpna : 누락된 데이터가 있는 축(로우, 컬럼)을 제외시킨다. 어느 정도의 누락 데이터까지 용인할 것인지 지정할 수 있다.
> - fiina : 누락된 데이터를 대신할 값을 채우거나 'ffill'이나 'bfill' 같은 보간 메서드를 적용한다.
> - isnull : 누락되거나 NA인 값을 알려주는 불리언값이 저장된 같은 형의 객체를 반환한다.
> - notnull : isnull과 반대되는 메서드

## 7.1.1 누락된 데이터 골라내기

- dropna 메서드를 적용하면 널이 아닌 non-null 데이터와 색인값만 들어 있는 Series를 반환한다.



In [7]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])

In [8]:
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [9]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [10]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [11]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


- dropna는 기본적으로 NA 값을 하나라도 포함하고 있는 로우를 제외시킨다.

In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [14]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [16]:
df = pd.DataFrame(np.random.randn(7, 3))

In [17]:
df

,0,1,2
0,-0.204708,0.478943,-0.519439
1,-0.555730,1.965781,1.393406
2,0.092908,0.281746,0.769023
3,1.246435,1.007189,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [18]:
df.iloc[:4, 1] = NA

In [19]:
df

,0,1,2
0,-0.204708,NaN,-0.519439
1,-0.555730,NaN,1.393406
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [20]:
df.iloc[:2, 2] = NA

In [21]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [22]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


- 몇 개 이상의 값이 들어 있는 로우만 살펴보고 싶다면 thresh 인자에 원하는 값을 넘기면 된다.

In [23]:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


## 7.1.2 결측치 채우기

- fillna 메서드를 활용해서, fillna 메서드에 채워 넣고 싶은 값을 넘겨주면 된다.

In [24]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


- fillna에 사전값을 넘겨서각 컬럼마다 다른 값을 채울 수도 있다.

In [25]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


- fillna는 새로운 객체를 반환하지만, 기존 객체를 변경할 수도 있다.

In [26]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [27]:
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [28]:
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.000940,1.343810
3,-0.713544,-0.831154,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512


In [29]:
df.iloc[2:, 1] = NA

In [30]:
df.iloc[4:, 2] = NA

In [31]:
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [32]:
df.fillna(method='ffill')

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [33]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


- Series의 평균값이나 중간값을 전달할 수도 있다.

In [34]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

 > fillna 함수 인자
 > - value : 비어 있는 값을 채울 스칼라값이나 사전 형식의 객체
 > - method : 보간 방식. 기본적으로 'ffill'을 사용한다.
 > - axis : 값을 채워 넣을 축. 기본값은 axis=0이다.
 > - inplace : 복사본을 생성하지 않고 호출한 객체를 변경한다. 기본값은 False다.
 > - limit : 값을 앞 혹은 뒤에서부터 몇 개까지 채울지 지정한다.

## 7.2 데이터 변형



### 7.2.1 중복 제거하기

In [35]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


- duplicated 메서드는 각 로우가 중복인지 아닌지 알려주는 불리언 Series를 반환한다.

In [36]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

- drop_duplicates는 duplicated 배열이 False인 DataFrame을 반환한다.

In [37]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [38]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


- 'k1' 컬럼에 기반해서 중복을 걸러내면 다음과 같이 한다.

In [39]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


- duplicates와 drop_duplicates는 기본적으로 처음 발견된 값을 유지한다. keep='last' 옵션을 넘기면 마지막으로 발견된 값을 반환한다.

In [40]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 7.2.2 함수나 매핑을 이용해서 데이터 변형하기

In [41]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [42]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [43]:
lowercased = data['food'].str.lower()
# 소문자로 변경
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [44]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [45]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.2.3 값 치환하기




In [86]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [87]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [88]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

- 두 개의 리스트 대신 사전을 이용하는 것도 가능하다.

In [89]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [50]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

### 7.2.4 축 색인 이름 바꾸기

In [91]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [92]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [93]:
transform = lambda x: x[:4].upper()

In [94]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [95]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


- 원래 객체를 변경하지 않고, 새로운 객체를 생성하려면 rename 메서드를 사용.

In [96]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


- rename메서드는 사전 형식의 객체를 이용해서 축 이름 중 일부만 변경하는 것도 가능.

In [97]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


- rename 메서드를 사용해서 DataFrame을 복사해서 index와 columns 속성을 갱신할 필요 없이 바로 변경 가능.
- 원본 데이터를 바로 변경하려면 inplace=True 옵션 사용.

In [99]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 개별화와 양자화

In [100]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

- pandas의 cut 함수르 이용해서 18-25, 26-35, 35-60, 60-100 그룹으로 나눈다.

In [101]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [102]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [103]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [105]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [106]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

- (}으로 간격을 나타내는 표기법이다. 중괄호로 시작해서 대괄호로 끝나는데, **중괄호 쪽의 값은 포함하지 않고 대괄호 쪽의 값은 포함하는** 간격을 나타낸다.
- right=False를 넘겨서 중괄호 대신 대괄호 쪽이 포함되지 않도록 바꿀 수 있다.

In [108]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [109]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)
#precision=2 은 소수점 아래 2자리까지로 제한.

[(0.34, 0.55], (0.34, 0.55], (0.76, 0.97], (0.76, 0.97], (0.34, 0.55], ..., (0.34, 0.55], (0.34, 0.55], (0.55, 0.76], (0.34, 0.55], (0.12, 0.34]]
Length: 20
Categories (4, interval[float64]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

In [110]:
data = np.random.randn(1000) # 정규분포

In [111]:
cats = pd.qcut(data, 4)  # 4분위로 분류
cats

[(-0.0265, 0.62], (0.62, 3.928], (-0.68, -0.0265], (0.62, 3.928], (-0.0265, 0.62], ..., (-0.68, -0.0265], (-0.68, -0.0265], (-2.9499999999999997, -0.68], (0.62, 3.928], (-0.68, -0.0265]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.68] < (-0.68, -0.0265] < (-0.0265, 0.62] < (0.62, 3.928]]

- qcut은 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있다.

In [112]:
pd.value_counts(cats) # qcut을 이용해서 4분위로 분류했다.

(-2.9499999999999997, -0.68]    250
(-0.68, -0.0265]                250
(-0.0265, 0.62]                 250
(0.62, 3.928]                   250
dtype: int64

In [113]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0265, 1.286], (-0.0265, 1.286], (-1.187, -0.0265], (-0.0265, 1.286], (-0.0265, 1.286], ..., (-1.187, -0.0265], (-1.187, -0.0265], (-2.9499999999999997, -1.187], (-0.0265, 1.286], (-1.187, -0.0265]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0265] < (-0.0265, 1.286] < (1.286, 3.928]]

- cut함수와 유사하게 변위치를 직접 지정해줄 수 있다.(변위치는 0부터 1까지)

### 7.2.6 특잇값을 찾고 제외하기

In [114]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.049091,0.026112,-0.002544,-0.051827
std,0.996947,1.007458,0.995232,0.998311
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


In [118]:
col = data[2]
col[np.abs(col) > 3]

41    -3.399312
136   -3.745356
Name: 2, dtype: float64

In [116]:
data[2]

0     -0.612905
1      0.434304
2      0.339231
3      2.406483
4     -0.395813
         ...   
995    1.229042
996    0.409304
997   -0.345727
998    1.128420
999   -2.314042
Name: 2, Length: 1000, dtype: float64

- 절댓값이 3을 초과하는 값이 들어 있는 모든 로우를 선택하려면 불리언 DataFrame에서 any 메서드를 사용하면 된다.

In [119]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
41,0.457246,-0.025907,-3.399312,-0.974657
60,1.951312,3.260383,0.963301,1.201206
136,0.508391,-0.196713,-3.745356,-1.520113
235,-0.242459,-3.056990,1.918403,-0.578828
258,0.682841,0.326045,0.425384,-3.428254
322,1.179227,-3.184377,1.369891,-1.074833
544,-3.548824,1.553205,-2.186301,1.277104
635,-0.578093,0.193299,1.397822,3.366626
782,-0.207434,3.525865,0.283070,0.544635
803,-3.645860,0.255475,-0.549574,-1.907459


In [120]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.050286,0.025567,-0.001399,-0.051765
std,0.992920,1.004214,0.991414,0.995761
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


- np.sign(data)는 data 값이 양수인지 음수인지에 따라 1이나 -1이 담긴 배열을 반환한다.

In [121]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,1.0
1,1.0,-1.0,1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,-1.0,-1.0,1.0,-1.0
4,-1.0,1.0,-1.0,-1.0


### 7.2.7 치환과 임의 샘플링

In [124]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [130]:
sampler = np.random.permutation(5)
sampler

array([4, 1, 0, 3, 2])

- 순서를 바꾸고 싶은 만큼의 길이를 permutation 함수로 넘기면 바뀐 순서가 담긴 정수 배열이 생성된다.

In [127]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [128]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


In [131]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11


- 치환 없이 일부만 임의로 선택하려면 Series나 DataFrame의 sample 메서드를 사용하면 된다.

In [132]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
0,0,1,2,3
4,16,17,18,19


- 표본을 치환을 통해 생성해내려면 sample에 replace=True 옵션을 넘긴다.

In [133]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
4    4
0    5
4    4
4    4
4    4
3    6
4    4
1    7
0    5
dtype: int64

### 7.2.8 표시자/더미 변수 계산하기

- 통계 모델이나 머신러닝 애플리케이션을 위해 분류값을 '더미'나 '표시자'행렬로 전환하는것.

In [134]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


- DataFrame 안에 있는 컬럼에 접두어(prefix)를 추가하고 싶을때.
    - get_dummies함수의 prefix 인자를 사용하면 이를 수행할 수 있다.

In [135]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


*********************************************

In [138]:
import os

path = os.path.join(os.getenv('HOME'),'aiffel/datasets/movielens')
os.chdir(path)

In [140]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames)
movies[:10]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [141]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [142]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [143]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [144]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [145]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [146]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

In [147]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [148]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                             1.0
Genre_Children's                            1.0
                               ...             
Genre_War                                   0.0
Genre_Musical                               0.0
Genre_Mystery                               0.0
Genre_Film-Noir                             0.0
Genre_Western                               0.0
Name: 0, Length: 21, dtype: object

### 이해가 안됨...

- get_dummies와 cut 같은 이산함수를 잘 조합하면 통계 애플리케이션에서 유용하게 사용가능하다.
**********************************************


In [149]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [150]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [151]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3 문자열 다루기

### 7.3.1 문자열 객체 메서드

In [152]:
val = 'a,b,  guido'
val.split(',')
# 쉼표로 구분된 문자열은 split 메서드를 이용해서 분리할 수 있다.

['a', 'b', '  guido']

- split 메서드는 공백 문자(줄바꿈 문자 포함)를 제거하는 strip 메서드와 조합해서 사용.

In [153]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [154]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [155]:
pieces

['a', 'b', 'guido']

In [156]:


'::'.join(pieces)



'a::b::guido'

In [157]:
'guido' in val

True

In [158]:
val.index(',')

1

In [159]:
val.find(':')

-1

- find와 index의 차이점은 index의 경우 문자열을 찾지 못하면 예외를 발생.
    - find의 경우에는 -1을 반환.

In [160]:
val.find(',')

1

In [161]:


val.index(':')



ValueError: substring not found

In [162]:
val.count(',')

2

- count는 특정 부분문자열이 몇 개 발견되었는지.

In [163]:
val.replace(',', '::')

'a::b::  guido'

- replace는 찾아낸 패턴을 다른 문자열로 변환.

In [164]:
val.replace(',', '')

'ab  guido'

> 파이썬 내장 문자열 함수

> - count : 문자열에서 겹치지 않는 부분문자열의 개수를 반환한다.
> - endswith : 문자열이 주어진 접미사로 끝날 경우 True를 반환한다.
> - startswith : 문자열이 주어진 접두사로 시작할 경우 True를 반환한다.
> - join : 문자열을 구분자로 하여 다른 문자열을 순서대로 이어붙인다.
> - index : 부분문자열의 첫 번째 글자의 위치를 반환한다. 부분문자열이 없을 경우 ValueError 예외가 발생한다.
> - find : 첫 번째 부분문자열의 첫 번째 글자의 위치를 반환한다. index와 유사하지만 부분 문자열이 없을 경우 -1을 반환한다.
> - rfind : 마지막 부분문자열의 첫 번째 글자의 위치를 반환한다. 부분문자열이 없을 경우 -1을 반환한다.
> - replace : 문자열을 다른 문자열로 치환한다.
> - strip, rstrip, lstrip : 개행 문자를 포함한 공백 문자를 제거한다. lstrip은 문자열의 시작 부분에 있는 공백 문자만 제거하며, rstrip은 문자열의 마지막 부분에 있는 공백 문자만 제거한다.
> - split : 문자열을 구분자를 기준으로 부분문자열의 리스트로 분리한다.
> - lower : 알파벳 문자를 소문자로 변환한다.
> - upper : 알파벳 문자를 대문자로 변환한다.
> - casefold : 문자를 소문자로 변환한다. 지역문자들은 그에 상응하는 대체 문자로 교체된다.
> - ljust, rjust : 문자열을 오른쪽 또는 왼쪽으로 정렬하고 주어진 길이에서 문자열의 길이를 제외한 나머지 부분은 공백 문자를 채워 넣는다.

### 7.3.2 정규 표현식

- 흔히 regex라 불리는 단일 표현식은 정규 표현 언어로 구성된 문자열이다. 파이썬에는 re 모듈이 내장되어 있어서 문자열에 대한 정규 표현식을 처리한다.

    - re 모듈 함수는 패턴 매칭, 치환, 분리 세가지로 나눌 수 있다.

In [165]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

- 먼저 정규표현식이 컴파일 되고 그다음에 split 메서드가 실행.

In [166]:
# re.compile로 직접 정규 표현식을 컴파일하고 얻은 정규 표현식 객체를 재사용 가능.
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [167]:
regex.findall(text)

['    ', '\t ', '  \t']

- 정규 표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall메서드를 사용.

### 정규 표현식에서 \문자가 이스케이프되는 문제를 피하려면 raw 문자열 표기법을 사용한다. 일반 문자로 처리하기 때문에 \를 간단하게 표현할 수 있다. 

- **즉, 'C:\\x'대신 r'C:\x'사용**.

In [168]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE는 정규 표현식이 대소문자를 가리지 않도록 한다.
regex = re.compile(pattern, flags=re.IGNORECASE)

- match와 search는 findall 메서드와 관련이 있다.
    - findall은 문자열에서 일치하는 모든 부분문자열을 찾아주지만 search 메서드는 패턴과 일치하는 첫 번째 존재를 반환한다. 
    - match 메서드는 이보다 더 엄격해서 문자열의 시작 부분에서 일치하는 것만 찾아준다.

In [169]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [170]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

- search는 텍스트에서 첫 번째 이메일 주소만을 찾아준다. 위 정규 표현식에 대한 match 객체는 그 정규 표현 패턴이 문자열 내에서 위치하는 시작점과 끝점만을 알려준다.

In [171]:
text[m.start():m.end()]

'dave@google.com'

In [172]:
print(regex.match(text))

None


- 정규 표현 패턴이 문자열의 시작점에서부터 일치하는지 검사하기 때문에 None 반환.

In [173]:
print(regex.sub('REDACTED', text))
# sub메서드는 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환한다.

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



- 이메일 주소를 찾아서 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사 세 가지 컴포넌트로 나눠야 한다면 각 패턴을 괄호로 묶어준다.

In [174]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [175]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [176]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

- 각 패턴 그룹에 접근 가능
    - \1은 첫 번째로 찾은 그룹, \2는 두 번째로 찾은 그룹.

In [177]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



> 정규 표현식 메서드
> - findall : 문자열에서 겹치지 않는 모든 발견된 패턴을 리스트로 반환한다.
> - finditer : findall과 같지만 발견된 패턴을 이터레이터를 통해 하나씩 반환한다.
> - match : 문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눈다. 일치하는 패턴이 있다면 match 객체를 반환하고 그렇지 않으면 None을 반환한다.
> - search : 문자열에서 패턴과 일치하는 내용을 검색하고 match 객체를 반환한다. match 메서드와는 다르게 시작부터 일치하는 내용만 찾지 않고 문자열 어디든 일치하는 내용이 있다면 반환한다.
> - split : 문자열에서 패턴과 일치하는 부분을 분리한다.
> - sub,subn : 문자열에서 일치하는 모든 패턴(sub) 혹은 처음 n개의 패턴(subn)을 대체 표현으로 치환한다. 대체 표현 문자열은 \1, \2,... 와 같은 기호를 사용해서 매치 그룹의 요소를 참조한다.


### 7.3.3 pandas의 벡터화된 문자열 함수

In [178]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [179]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

- 문자열과 정규 표현식 메서드는 data.map을 사용해서 각 값에 적용(lambda 혹은 다른 함수를 넘겨서)할 수 있지만 NA 값을 만나면 실패하게 된다.

In [180]:
data.str.contains('gmail')
# contains 함수를 통해 포함하고 있는지 검사 가능

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [181]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [182]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [183]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [184]:
matches.str.get(1)

AttributeError: Can only use .str accessor with string values!

In [185]:
matches.str[0]

AttributeError: Can only use .str accessor with string values!

In [186]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [187]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

> 벡터화된 문자열 메서드
> - cat : 선택적인 구분자와 함께 요소별로 문자열을 이어붙인다.
> - contains  : 문자열이 패턴이나 정규 표현식을 포함하는지 나타내는 불리언 배열을 반환한다.
> - count : 일치하는 패턴 수를 반환한다.
> - extract : 문자열이 담긴 Series에서 하나 이상의 문자열을 추출하기 위해 정규 표현식을 이용한다. 결과는 각 그룹이 하나의 컬럼이 되는 DataFrame이다.
> - endswith : 각 요소에 대해 x.endswith(pattern)과 동일한 동작을 한다.
> - startswith : 각 요소에 대해 x.startswith(pattern)과 동일한 동작을 한다.
> - findall : 각 문자열에 대해 일치하는 패턴/정규 표현식의 전체 목록을 구한다.
> - get : i번째 요소를 반환한다.
> - isalnum : 내장 함수 str.isalnum과 동일
> - isalpha : 내장 함수 str.isalpha와 동일
> - isdecimal : 내장 함수 str.isdecimal과 동일
> - isdigit : 내장 함수 str.isdigit와 동일
> - islower : 내장 함수 str.islower와 동일
> - isnumeric : 내장 함수 str.isnumeric과 동일
> - isupper : 내장함수 str.isupper과 동일
> - join : Series의 각 요소를 주어진 구분자로 연결한다.
> - len : 각 문자열의 길이를 구한다.
> - lower, upper : 대소문자로 변환한다. 각 요소에 대한 x.lower(), x.upper()와 같다.
> - match : 주어진 정규 표현식으로 각 요소에 대한 re.match를 수행하여 일치하는 그룹을 리스트로 반환한다.
> - pad : 문자열의 좌, 우 혹은 양쪽에 공백을 추가한다.
> - center  : pad(side='both')와 동일
> - repeat : 값을 복사한다. 예를 들어 s.str.repeat(3)은 각 문자열에 대한 x*3과 동일하다.
> - replace : 패턴/정규표현식과 일치하는 내용을 다른 문자열로 치환한다.
> - slice : Series 안에 있는 각 문자열을 자른다.
> - split : 정규 표현식 혹은 구분자로 문자열을 나눈다.
> - strip : 개행 문자를 포함하여 왼쪽과 오른쪽의 공백 문자를 제거한다.
> - rstrip : 오른쪽의 공백 문자를 제거한다.
> - lstrip : 왼쪽의 공백 문자를 제거한다.